In [2]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm
import re

In [5]:
import re
p = re.compile('ab*')
p # re.compile(r'ab*', re.UNICODE)

re.compile(r'ab*', re.UNICODE)

In [6]:
re.UNICODE.value

32

In [29]:
p = re.compile('[a-t]+')
m = p.match("temupau")
print(m) # <re.Match object; span=(0, 3), match='tem'>

<re.Match object; span=(0, 3), match='tem'>


In [32]:
m.group() # 'tem'
m.start() # 0
m.end()   # 3
m.span()  # (0, 3)

(0, 3)

In [39]:
print(p.match('::: message')) # None
m = p.search('::: message'); print(m) #<re.Match object; span=(4, 11), match='message'>
m.group() #'message'
m.start() # 4
m.end() # 11
m.span() # (4, 11)

None
<re.Match object; span=(4, 11), match='message'>


(4, 11)

In [44]:
p = re.compile(r'[a-r]+')
m = p.match('ramesh tau')
if m:
    print('Matched', m.group())
else:
    print('No Match')
## Output - Matched rame

Matched rame


In [45]:
p = re.compile(r'\d+')
p.findall('12 drummers drumming, 11 pipers piping, 10 lords a-leaping')

['12', '11', '10']

In [49]:
p = re.compile(r'\d+')
iterator = p.finditer('12 drummers drumming, 11 pipers piping, 10 lords a-leaping')
for match in iterator:
    print(match.group())
    print(match.start())
    print(match.end())
    print(match.span())

12
0
2
(0, 2)
11
22
24
(22, 24)
10
40
42
(40, 42)


In [50]:
p = re.compile(r'[a-m]')
p.search('allubro')

<re.Match object; span=(0, 1), match='a'>

In [4]:
print(re.match(r'From\s+', 'Fromage amk')) # None
print(re.match(r'From\s+', 'From 7 till 10')) # <re.Match object; span=(0, 5), match='From '>

None
<re.Match object; span=(0, 5), match='From '>


In [6]:
print(re.search(r'^From', 'From here to eternity')) # <re.Match object; span=(0, 4), match='From'>
print(re.search(r'^From', 'Reciting From Memory')) # None

<re.Match object; span=(0, 4), match='From'>
None


In [11]:
p = re.compile('\bclass\b')
print(p.search('This is my class')) # None
print(p.search('THis is declassified')) # None
print(p.search('One subclass at a time')) # None
print(p.search('\b' + 'class' + '\b')) # <re.Match object; span=(0, 7), match='\x08class\x08'>

None
None
None
<re.Match object; span=(0, 7), match='\x08class\x08'>


In [17]:
p = re.compile('(ab)*')
print(p.match('ababbbababb').span(1)) # (2, 4)

(2, 4)


In [23]:
p = re.compile('(a)b')
print(p.match('ab')) # <re.Match object; span=(0, 2), match='ab'>
m = p.match('ab')
print(m.group(0)) # ab
print(m.group(1)) # a

<re.Match object; span=(0, 2), match='ab'>
ab
a


In [30]:
p = re.compile('(a(b)c)d')
m = p.match('abcd')
m.group() # 'abcd'
m.group(1) # 'abc'
m.group(2) # b
m.group(2,1,2) # ('b', 'abc', 'b')
m.groups() # ('abc', 'b')

('abc', 'b')

In [38]:
p = re.compile(r'\b(\w+)\s+\1\b')
p.search('This is the the Paris').group() # 'the the'

'the the'

In [40]:
m = re.match("([abc])+", "abc")
m.groups() # ('c',)
m = re.match("(?:[abc])+", "abc")
m.groups() # ()

()

In [45]:
p = re.compile(r'\W+')
p.split('This is a test, short and sweet, of split().')
p.split('This is a test, short and sweet, of split().', 3)


['This', 'is', 'a', 'test, short and sweet, of split().']

In [46]:
p = re.compile(r'\W+')
p2 = re.compile(r'(\W+)')

p.split('This.... is a test.') # ['This', 'is', 'a', 'test', '']
p2.split('This.... is a test.') # ['This', '.... ', 'is', ' ', 'a', ' ', 'test', '.', '']

['This', '.... ', 'is', ' ', 'a', ' ', 'test', '.', '']

In [48]:
p = re.compile('(blue|white|red)', re.IGNORECASE)
p.sub('color', 'blue shocks and Red shoes', count=1) # 'color shocks and Red shoes'

'color shocks and Red shoes'

In [53]:
p = re.compile('(blue|white|red)', re.IGNORECASE)
p.subn('color', 'blue and red')
p.subn('color', 'blue and red', count=1)

('color and red', 1)

In [55]:
p = re.compile(r'x*')
p.sub('-', 'abxd cx')

'-a-b--d- -c--'

In [57]:
p = re.compile('section{ ( [^}]* ) }', re.VERBOSE)
p.sub(r'subsection{\1}','section{First} section{second}')

'subsection{First} subsection{second}'